In [1]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import numpy as np
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import torch
from torch import nn
import pandas as pd
from torch.autograd import Variable
from Models import GateCNNFahsionSoftmax, CNNFashion, GateCNNFashion
from Util import FedAvg, Preparedata, Sample_node
from torch.utils.data import DataLoader, Dataset
from Update import ClientUpdate

import torch
from torch import nn, autograd

In [2]:
trans_fashionmnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset_train = datasets.FashionMNIST('../data/fashion-mnist', train=True, download = True, transform = trans_fashionmnist)
dataset_test = datasets.FashionMNIST('../data/fashion-mnist',train=False, download = True, transform = trans_fashionmnist)

In [3]:
num_clients = 4
ntrain = 1000
ntest = 300
train_sets = []
test_sets = []
percent = 0.8
#creat no-iid data
# for i in range(num_clients):
#     train_set, test_set = Preparedata(dataset_train, dataset_test, ntrain, ntest, class_number = [i%10], percent = percent)
#     train_sets.append(train_set)
#     test_sets.append(test_set)

In [4]:
train_set, test_set = Preparedata(shuffle(dataset_train), dataset_test, ntrain, ntest, class_number = [0,1,2], percent = percent)
train_sets.append(train_set)
test_sets.append(test_set)


train_set, test_set = Preparedata(dataset_train, dataset_test, ntrain, ntest, class_number = [0,1,2], percent = percent)
train_sets.append(train_set)
test_sets.append(test_set)


train_set, test_set = Preparedata(shuffle(dataset_train), dataset_test, ntrain, ntest, class_number = [5,6,8], percent = percent)
train_sets.append(train_set)
test_sets.append(test_set)


train_set, test_set = Preparedata(dataset_train, dataset_test, ntrain, ntest, class_number = [5,6,8], percent = percent)
train_sets.append(train_set)
test_sets.append(test_set)

In [5]:
class ClientUpdate(object):
    def __init__(self, train_set=None,  test_set=None, idx=None):
        
        self.loss_func = nn.NLLLoss()
        
        self.train_set, self.val_set = torch.utils.data.random_split(train_set, [800, 200], torch.Generator().manual_seed(idx))

        self.ldr_train = DataLoader(self.train_set, batch_size=10, shuffle=True)
        self.ldr_val = DataLoader(self.val_set, batch_size = 1, shuffle=True)

        self.test_set = test_set
        self.ldr_test = DataLoader(self.test_set, batch_size = 1, shuffle=True)
    
    def train(self, net, n_epochs,learning_rate):
        
        net.train()
        
        optimizer = torch.optim.Adam(net.parameters(),lr=learning_rate)

        epoch_loss = []

        for iter in range(n_epochs):
            net.train()
            batch_loss = []
            
            for batch_idx, (images, labels) in enumerate(self.ldr_train):
                images, labels = (images, labels)
                net.zero_grad()
                log_probs = net(images.float())
    
                loss = self.loss_func(log_probs, labels)
                loss.backward()
                optimizer.step()
                
                batch_loss.append(loss.item())
                
            epoch_loss.append(sum(batch_loss)/len(batch_loss))
            
#             val_acc, val_loss = self.validate(net,True)
#             print(val_acc)

        return net.state_dict(), epoch_loss[-1]
   
    
    def train_finetune(self, net, n_epochs, learning_rate):
        net.train()
        
        optimizer = torch.optim.Adam(net.parameters(),lr=learning_rate)
        
        patience = 10
        epoch_loss = []
        epoch_train_accuracy = []
        model_best = net.state_dict()
        train_acc_best = np.inf
        val_acc_best = -np.inf
        val_loss_best = np.inf
        counter = 0
        
        for iter in range(n_epochs):
            net.train()
            batch_loss = []
            correct = 0
            
            for batch_idx, (images, labels) in enumerate(self.ldr_train):
                images, labels = (images, labels)
                net.zero_grad()
                log_probs = net(images.float())
                loss = self.loss_func(log_probs, labels)
                loss.backward()
                optimizer.step()
                batch_loss.append(loss.item())
                _, predicted = torch.max(log_probs.data, 1)
                correct += (predicted == labels).sum().item()
            train_accuracy = 100.00 * correct / len(self.ldr_train.dataset)
            epoch_train_accuracy.append(train_accuracy)
            epoch_loss.append(sum(batch_loss)/len(batch_loss))
            
            if(iter%5==0):
                val_acc, val_loss = self.validate(net,True)
                net.train()
                if(val_loss < val_loss_best - 0.01):
                    counter = 0
                    model_best = net.state_dict()
                    val_acc_best = val_acc
                    val_loss_best = val_loss
                    train_acc_best = train_accuracy
                    #print("Iter: %d | %.2f" %(iter,val_acc_best))
                else:
                    counter = counter+1
                    
                # early stop
                if counter == patience:
                    return model_best, epoch_loss[-1], val_acc_best, train_acc_best
                    
    
        return model_best, epoch_loss[-1], val_acc_best, train_acc_best
     
        
    def train_mix(self, net_local, net_trans, gate, train_gate_only, n_epochs, early_stop, learning_rate, val):

        print("start train mix model")        
        gate.train()
        net_local.train()
        net_trans.train()

        if(train_gate_only):
            optimizer = torch.optim.Adam(gate.parameters(),lr=learning_rate)
        else:
            optimizer = torch.optim.Adam(list(gate.parameters())+list(net_local.parameters()),lr=learning_rate)

      
        patience = 30
        epoch_loss = []
        gate_best = gate.state_dict()
        local_best = net_local.state_dict()
        trans_best = net_trans.state_dict()
        val_acc_best = -np.inf
        val_loss_best = np.inf
        counter = 0
        gate_values_best = 0

        
        for iter in range(n_epochs):
            net_local.train()
            net_trans.train()
            gate.train()

            batch_loss = []

            for batch_idx, (images, labels) in enumerate(self.ldr_train):
                images, labels = (images, labels)

                net_local.zero_grad()
                net_trans.zero_grad()
                gate.zero_grad()

                gate_weight = gate(images.float())
            

                # gate_weight*wi + gate_weight*fintuned
                local_prob = gate_weight*net_local(images.float())+(1-gate_weight)*net_trans(images.float())
                loss = self.loss_func(local_prob,labels)
  
                loss.backward()
                optimizer.step()
                batch_loss.append(loss.item())

            epoch_loss.append(sum(batch_loss)/len(batch_loss))
            
            
            if(early_stop):
                if(iter%5==0):
                    val_acc, val_loss = self.validate_mix(net_local, net_trans, gate, val)
                    net_local.train()
                    net_trans.train()
                    gate.train()

                    if(val_loss < val_loss_best - 0.01):
                        counter = 0
                        gate_best = gate.state_dict()
                        local_best = net_local.state_dict()
                        trans_best = net_trans.state_dict()
                        val_acc_best = val_acc
                        val_loss_best = val_loss
                        gate_weight_best = gate_weight
                        print("Iter: %d | %.2f" %(iter,val_acc_best))
                    else:
                        counter = counter + 1
                        #print(counter)
                
                if counter == patience:
                    return gate_best, local_best, trans_best, epoch_loss[-1],val_acc_best, sum(gate_weight_best) / len(gate_weight_best)


        return gate_best, local_best, trans_best, epoch_loss[-1],val_acc_best, sum(gate_weight_best) / len(gate_weight_best)
    
        
    def train_mix_clients(self, net_local, net_trans, net_clients, gate_frezze, gate, train_gate_only, n_epochs, early_stop, learning_rate, val):

        print("start train mix model")        
        gate.train()
        net_local.train()
        net_trans.train()

        if(train_gate_only):
            optimizer = torch.optim.Adam(gate.parameters(),lr=learning_rate)
        else:
            optimizer = torch.optim.Adam(list(gate.parameters())+list(net_local.parameters()),lr=learning_rate)

      
        patience = 30
        epoch_loss = []
        gate_best = gate.state_dict()
        local_best = net_local.state_dict()
        trans_best = net_trans.state_dict()
        val_acc_best = -np.inf
        val_loss_best = np.inf
        counter = 0
        gate_values_best = 0

        
        for iter in range(n_epochs):
            net_local.train()
            net_trans.train()
            gate.train()

            batch_loss = []

            for batch_idx, (images, labels) in enumerate(self.ldr_train):
                images, labels = (images, labels)

                net_local.zero_grad()
                net_trans.zero_grad()
                gate.zero_grad()

                gate_inner = gate_frezze(images.float())
                gate_outer = gate(images.float())
             

                # gate_weight*wi + gate_weight*fintuned
                local_prob_inner = gate_inner*net_local(images.float())+(1-gate_inner)*net_trans(images.float())
                local_prob = gate_outer*net_clients(images.float())+(1-gate_outer)*local_prob_inner
                loss = self.loss_func(local_prob,labels)
  
                loss.backward()
                optimizer.step()
                batch_loss.append(loss.item())

            epoch_loss.append(sum(batch_loss)/len(batch_loss))
            
            
            if(early_stop):
                if(iter%5==0):
                    val_acc, val_loss = self.validate_mix_clients(net_local, net_trans, net_clients, gate_frezze, gate, val)
                    net_local.train()
                    net_trans.train()
                    gate.train()

                    if(val_loss < val_loss_best - 0.01):
                        counter = 0
                        gate_best = gate.state_dict()
                        local_best = net_local.state_dict()
                        trans_best = net_trans.state_dict()
                        val_acc_best = val_acc
                        val_loss_best = val_loss
                        gate_weight_best = weight
                        print("Iter: %d | %.2f" %(iter,val_acc_best))
                    else:
                        counter = counter + 1
                        #print(counter)
                
                if counter == patience:
                    return gate_best, local_best, trans_best, epoch_loss[-1],val_acc_best, gate_weight_best


        return gate_best, local_best, trans_best, epoch_loss[-1],val_acc_best, gate_outer
    
    
    def validate(self,net,val):
        # if true validate dataset, if false use test detaset
        if(val):
            dataloader = self.ldr_val
        else:
            dataloader = self.ldr_test
       
        with torch.no_grad():
            net.eval()
            # validate
            val_loss = 0
            correct = 0
            for idx, (data, target) in enumerate(dataloader):
                data, target = (data, target)
                log_probs = net(data.float())

                val_loss += self.loss_func(log_probs, target).item()

                y_pred = log_probs.data.max(1, keepdim=True)[1]
                correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()

            val_loss /= len(dataloader.dataset)
            accuracy = 100.00 * correct / len(dataloader.dataset)
   
        return accuracy.item(), val_loss
    
    def validate_mix(self, net_l, net_t, gate, val):
        # if true validate dataset, if false use test detaset
        if(val):
            dataloader = self.ldr_val
        else:
            dataloader = self.ldr_test
        
        with torch.no_grad():
            net_l.eval()
            net_t.eval()
            gate.eval()
            val_loss = 0
            correct = 0
            gate_values = np.array([])
            label_values = np.array([])
            
            for idx, (data,target) in enumerate(dataloader):
                data, target = (data,target)
                gate_weight = gate(data.float())
                
                log_prob = gate_weight*net_l(data.float())+(1-gate_weight)*net_t(data.float())
                

                val_loss += self.loss_func(log_prob,target).item()
                y_pred = log_prob.data.max(1,keepdim=True)[1]
                correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()

        val_loss /= len(dataloader.dataset)
        accuracy = 100.00 * correct / len(dataloader.dataset)
        return accuracy.item(), val_loss
    
    
    
    def validate_mix_clients(self, net_l, net_t, net_c, gate_f, gate, val):
        # if true validate dataset, if false use test detaset
        if(val):
            dataloader = self.ldr_val
        else:
            dataloader = self.ldr_test
        
        with torch.no_grad():
            net_l.eval()
            net_t.eval()
            gate.eval()
            net_c.eval()
            val_loss = 0
            correct = 0
            gate_values = np.array([])
            label_values = np.array([])
            
            for idx, (data,target) in enumerate(dataloader):
                data, target = (data,target)
                gate_inner = gate_f(data.float())
                gate_outer = gate(data.float())
                
                local_prob_inner = gate_inner*net_l(data.float())+(1-gate_inner)*net_t(data.float())
                log_prob = gate_outer*net_c(data.float())+(1-gate_outer)*local_prob_inner

                val_loss += self.loss_func(log_prob,target).item()
                y_pred = log_prob.data.max(1,keepdim=True)[1]
                correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()

        val_loss /= len(dataloader.dataset)
        accuracy = 100.00 * correct / len(dataloader.dataset)
        return accuracy.item(), val_loss



In [6]:
num_classes = 10

In [7]:
net_locals = [] # save the local model of every client

for idx in range(num_clients):
    net_local = CNNFashion(num_classes)
    net_locals.append(net_local)

In [14]:
local_ep = 100
lr = 1e-4

w_fedAvg = []
alpha = []
train_loss = []
val_loss = []
val_acc = []
w_clients = []
net_fed = []
val_local = []
fintuned_acc = []

In [15]:
num_classes = 10
net_glob_fedAvg = CNNFashion(num_classes)

net_locals = []
for idx in range(num_clients):
    net_local = CNNFashion(num_classes)
    net_locals.append(net_local)
    
acc = []
w_fedAvg = []

In [16]:
G = np.ones( (num_clients, num_clients) )*(1/(num_clients-1))

for idx in range(num_clients):
    G[idx][idx] = 0

In [17]:
graph = np.zeros( (num_clients, num_clients) )

In [18]:
# for loop in range(5):
    # tain every client seperately
#     train_set = []
#     test_set = []
#     train_set, test_set = Preparedata(shuffle(dataset_train), dataset_test, ntrain, ntest, class_number = [0,1,2], percent = percent)
#     train_sets.append(train_set)
#     test_sets.append(test_set)


#     train_set, test_set = Preparedata(shuffle(dataset_train), dataset_test, ntrain, ntest, class_number = [0,1,2], percent = percent)
#     train_sets.append(train_set)
#     test_sets.append(test_set)


#     train_set, test_set = Preparedata(shuffle(dataset_train), dataset_test, ntrain, ntest, class_number = [5,6,8], percent = percent)
#     train_sets.append(train_set)
#     test_sets.append(test_set)


#     train_set, test_set = Preparedata(shuffle(dataset_train), dataset_test, ntrain, ntest, class_number = [5,6,8], percent = percent)
#     train_sets.append(train_set)
#     test_sets.append(test_set)
val_local = []
for idx in range(num_clients):
        print("Client %d" %(idx))


        client = ClientUpdate(train_set=train_sets[idx], test_set = test_sets[idx],
                                   idx = idx)

        # trian the ith w
        net_updated, train_loss_idx = client.train(net = net_locals[idx], n_epochs = 150,learning_rate = 1e-4)

        w_fedAvg.append(copy.deepcopy(net_updated))

        net_locals[idx].load_state_dict(w_fedAvg[idx])

        train_loss.append(train_loss_idx)

        # envaluate the local nets
        local_val_acc, val_loss = client.validate(net_locals[idx],False)
        val_local.append(local_val_acc)


print(val_local)
   
for loop in range(5):
    for idx in range(num_clients):
        for i in range(num_clients):
            if (i > idx):
                print("Client %d and % d" %(idx, i))
                w = [w_fedAvg[idx],w_fedAvg[i]]
                alpha = [0.5,0.5]
                w_global = FedAvg(w,alpha)
                net_glob_fedAvg.load_state_dict(w_global)


                fintuned_nets = []
                fintuned_acc = []
                for j in [idx,i]:
                        print("Client %d" %(j))


                        client = ClientUpdate(train_set=train_sets[j], test_set = test_sets[j],
                                                   idx = j)

                        # trian the ith w
                        net_w, train_loss_idx, val_acc_idx, train_acc_idx = client.train_finetune(net = net_glob_fedAvg, 
                                                                n_epochs = 150, learning_rate = 1e-4)

                        ft_net = copy.deepcopy(net_glob_fedAvg)
                        ft_net.load_state_dict(net_w)
                        fintuned_nets.append(ft_net)


                        fintuned_val_acc, val_loss = client.validate(ft_net,False)
                        fintuned_acc.append(fintuned_val_acc)

                print(fintuned_acc)

                if(fintuned_acc[0] > val_local[idx]+1):
                    graph[idx,i]=graph[idx,i]+1
                elif(fintuned_acc[0] < val_local[idx]-1):
                    graph[idx,i]=graph[idx,i]-1

                if(fintuned_acc[1] > val_local[i]+1):
                    graph[i,idx]=graph[i,idx]+1
                elif(fintuned_acc[1] < val_local[i]-1):
                    graph[i,idx]=graph[i,idx]-1


    print(val_local)         

Client 0
Client 1
Client 2
Client 3
[78.0, 79.0, 76.0, 76.66666412353516]
Client 0 and  1
Client 0
Client 1
[77.66666412353516, 83.66666412353516]
Client 0 and  2
Client 0
Client 2
[83.66666412353516, 80.66666412353516]
Client 0 and  3
Client 0
Client 3
[83.66666412353516, 80.33333587646484]
Client 1 and  2
Client 1
Client 2
[87.0, 62.66666793823242]
Client 1 and  3
Client 1
Client 3
[83.33333587646484, 61.0]
Client 2 and  3
Client 2
Client 3
[76.66666412353516, 77.66666412353516]
[78.0, 79.0, 76.0, 76.66666412353516]
Client 0 and  1
Client 0
Client 1
[78.0, 83.66666412353516]
Client 0 and  2
Client 0
Client 2
[83.66666412353516, 80.66666412353516]
Client 0 and  3
Client 0
Client 3
[84.0, 80.0]
Client 1 and  2
Client 1
Client 2
[86.33333587646484, 63.33333206176758]
Client 1 and  3
Client 1
Client 3
[84.66666412353516, 62.33333206176758]
Client 2 and  3
Client 2
Client 3
[76.66666412353516, 77.66666412353516]
[78.0, 79.0, 76.0, 76.66666412353516]
Client 0 and  1
Client 0
Client 1
[78.0

In [19]:
graph

array([[ 0.,  1.,  5.,  5.],
       [ 5.,  0.,  5.,  3.],
       [ 5., -1.,  0.,  1.],
       [ 5., -4.,  1.,  0.]])

In [20]:
# update weight according the graph
# the sum of every row quales one
for i in range(num_clients):
    for j in range(num_clients):
        if (graph[i][j]==5):
            G[i][j] = G[i][j]*1.3
        elif (graph[i][j]==4):
            G[i][j] = G[i][j]*1.2
        elif (graph[i][j]==3):
            G[i][j] = G[i][j]*1.1
        elif (graph[i][j]== -5):
            G[i][j] = G[i][j]/1.3
        elif (graph[i][j]== -4):
            G[i][j] = G[i][j]/1.2
        elif (graph[i][j]== -3):
            G[i][j] = G[i][j]/1.1

In [21]:
for idx in range(num_clients):
    G[idx] = [x / sum(G[idx]) for x in G[idx]]

In [22]:
G

array([[0.        , 0.27777778, 0.36111111, 0.36111111],
       [0.35135135, 0.        , 0.35135135, 0.2972973 ],
       [0.39393939, 0.3030303 , 0.        , 0.3030303 ],
       [0.41489362, 0.26595745, 0.31914894, 0.        ]])